### This is a simple "Hello World" example for MPI with iPython
For references, these websites are not terrible
- https://ipython.org/ipython-doc/3/parallel/magics.html
- https://charlesreid1.com/wiki/Jupyter/MPI

#### Before running this example, do the following
1) Install dependencies

        brew install open-mpi
        pip3 install mpi4py
        pip3 install ipyparallel
        jupyter serverextension enable --py ipyparallel

2)  Start local ipython cluster independently of jupyter
- Create mpi ipython profile

        $ ipython profile create --parallel --profile=mpi
   
- Note, MPI should be listed as installed in the file

        ~/.ipython/profile_mpi/ipcluster_config.py`


3) Start ipython cluster from new terminal

        $ ipcluster start --n=2 --engines=mpi --profile=mpi
   
or depending, may need to use
   
        $ ipcluster start --n=2 --engines=MPIEngineSetLauncher --profile=mpi

4) Leave that ipython cluster running in its own terminal
- Issues:  This page is helpful https://charlesreid1.com/wiki/Jupyter/MPI


5) Connect to ipython cluster in this Jupyter notebook by running the below code blocks
- Note how you need to first connect to the ipython cluster, and then synchronize imports of packages


In [2]:
# Connect to local ipython cluster.  Note, the ipcluster profile name must match with the below text. 
# Here, we use 'mpi', but you can name the cluster profile anything
from ipyparallel import Client, error
cluster = Client(profile='mpi')

# Note, how you synchronize package imports
with cluster[:].sync_imports():
    from mpi4py import MPI
    import sys

importing MPI from mpi4py on engine(s)
importing sys on engine(s)


In [3]:
# Diagnostic printing of ipcluster information
print('profile:', cluster.profile)
print("IDs:", cluster.ids) 

profile: mpi
IDs: [0, 1]


#### The command `%%px` tells iPython to execute all of these lines in parallel, on each iPython process
- Alternatively, prepending a line with `%px` will set individual lines to run in parallel
- Without line-by-line `%px` or `%%px`, commands will only run on rank 0
- We use sys.stderr to write output, as this is better in parallel.  This function always returns a dummy integer (representing the number of characters printed), hence the return value `k` below.

In [4]:
%%px 
size = MPI.COMM_WORLD.Get_size()
rank = MPI.COMM_WORLD.Get_rank()
k = sys.stderr.write("Hello, World! I am process %d of %d.\n"% (rank, size))

[stderr:0] Hello, World! I am process 0 of 2.


[stderr:1] Hello, World! I am process 1 of 2.
